In [2]:
import pandas as pd
import numpy as np
import regex as re

In [55]:
df = pd.read_csv('F508_F.csv')
df.sort_values(by=['Id Paciente'], inplace=True)
#df.dropna(subset=['Examen físico'], inplace=True)
df.shape

(618, 38)

In [56]:
df.columns

Index(['Unnamed: 0', 'Consulta', 'Fecha_inicio_consulta',
       'Hora_inicio_consulta', 'Fecha_fin_consulta', 'Hora_fin_consulta',
       'Nro. autorización', 'Id Paciente', 'Tipo Id', 'Sexo Paciente',
       'Código Departamento', 'Código Municipio', 'Nombre Municipio',
       'Fecha nacimiento', 'Edad actual', 'Edad paciente consulta',
       'EPS Paciente', 'id Médico', 'Diagnóstico_principal',
       'Todos_Diagnósticos', 'Servicio(s)', 'Descripción del servicio',
       'Finalidad', 'Causa externa', 'Observación Dx', 'Motivo consulta',
       'Enfermedad actual', 'Revisión sistemas', 'Antecedentes',
       'Examen físico', 'Opinión plan', 'Medicamentoque trae paciente',
       'Examenes que trae paciente', 'Notas aclaratorias', 'Duración Consulta',
       'Estado Consulta', 'Sedes', 'Modalidad de atención'],
      dtype='object')

In [58]:
def generic(word, column):
    output = []
    for sen in column.values:
        try:
            output.append(re.search(word + ':\s*(\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '\s*(\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '.\s*(\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '.:\s*(\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + ':.\s*(\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '\s*(\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            output.append(np.nan)
    return output

In [59]:
def generic_1(word, column):
    output = []
    for sen in column.values:
        try:
            output.append(re.search(word + ':\s*(\d+.\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '\s*(\d+.\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '.\s*(\d+.\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '.:\s*(\d+.\d)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + ':.\s*(\d+.\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            pass
        try:
            output.append(re.search(word + '\s*(\d+.\d+)', sen, re.IGNORECASE).group(1))
            continue
        except:
            output.append(np.nan)
    return output

## Talla y peso

In [60]:
df['Examen físico'] = df['Examen físico'].str.lower().str.strip()

In [61]:
df['Examen físico'].str.lower().str.strip().values[22]

'peso:8880 gramos talla: 77 cm pc: 47.4 cm orl bien cardiopulmnar. bien abdomen. bien no eccema'

In [62]:
result = np.array([generic_1('peso', df['Examen físico']), generic('talla', df['Examen físico'])])
result = np.stack(result).T

In [63]:
result.shape

(618, 2)

In [64]:
df_peso_talla = pd.DataFrame(result, columns=['peso', 'talla'])

In [65]:
df_peso_talla

,peso,talla
0,8310,71
1,82.5,178
2,35.7,143
3,18.4,114
4,9.8,82
...,...,...
613,8360,70
614,8.650,76
615,11.360,81
616,8.7,71


## Antecedentes

In [66]:
df['Antecedentes'] = df['Antecedentes'].str.lower().str.strip()

In [67]:
gestacional = []
for sentence in df.Antecedentes.values:
    try:
        gestacional.append(re.search('(\d+)\s*semanas', sentence, re.IGNORECASE).group(1))
    except:
        gestacional.append(np.nan)

In [68]:
tipo_parto = []
for sentence in df.Antecedentes.values:
    if type(sentence) == str:
        value = re.search('parto v', sentence, re.IGNORECASE)
    else:
        tipo_parto.append(np.nan)
        continue
    if value is None:
        value = re.search('cesarea', sentence, re.IGNORECASE)
        tipo_parto.append('Cesarea')        
    else:
        tipo_parto.append('Vaginal')
    

In [69]:
clas_eg = []
for sentence in df.Antecedentes.values:
    if type(sentence) == str:
        value = re.search('gestacion normal', sentence, re.IGNORECASE)
    else: 
        clas_eg.append(np.nan)
        continue
    if value is None:
        clas_eg.append('Pretermino')        
    else:
        clas_eg.append('Termino')

In [70]:
peso_nacer = np.array(generic('peso', df.Antecedentes))
peso_nacer[np.argwhere(~np.isnan(np.array(generic('peso al nacer', df.Antecedentes)).astype(float)))] = np.array(generic('peso al nacer', df.Antecedentes))[np.argwhere(~np.isnan(np.array(generic('peso al nacer', df.Antecedentes)).astype(float)))]

In [71]:
fumador = []
for sentence in df.Antecedentes.values:
    if type(sentence) == str:
        value = re.search('no es fumador', sentence, re.IGNORECASE)
    else: 
        fumador.append(np.nan)
        continue
    if value is None:
        value = re.search('no fumador', sentence, re.IGNORECASE)
        if value is None:
            value = re.search(r'\bfumador\b', sentence)
            if value is None:
                fumador.append(np.nan)    
            else:
                fumador.append('Es fumador')  
        else:
            fumador.append('No es fumador pasivo')
    else:
        fumador.append('No es fumador pasivo')

In [72]:
antecedentes_madre = []
for sentence in df.Antecedentes.values:
    if type(sentence) == str:
        value = re.search(r'af:\s*madre:\s*(.*?)\s*padre', sentence)
    else: 
        antecedentes_madre.append(np.nan)
        continue
    if value is None:
        value = re.search(r'af:\s*madre.\s*(.*?)\s*padre', sentence)
        if value is None:
            value = re.search(r'af:\s*madre.\s*(.*?)\s*padre', sentence)
            if value is None:
                value = re.search(r'af.\s*madre.\s*(.*?)\s*padre', sentence)
                if value is None:
                    value = re.search(r'af.\s*madre:\s*(.*?)\s*padre', sentence)
                    if value is None:
                        antecedentes_madre.append(np.nan)   
                    else:
                        antecedentes_madre.append(value.group(1)) 
                else:
                    antecedentes_madre.append(value.group(1))
        else:
            antecedentes_madre.append(value.group(1))
    else:
        antecedentes_madre.append(value.group(1))

In [73]:
antecedentes_padre = []
for sentence in df.Antecedentes.values:
    if type(sentence) == str:
        value = re.search(r'padre:\s*(.*?)$', sentence)
    else: 
        antecedentes_padre.append(np.nan)
        continue
    if value is None:
        value = re.search(r'padre.\s*(.*?)$', sentence)
        if value is None:
            value = re.search(r'padre\s*(.*?)$', sentence)
            if value is None:
                antecedentes_padre.append(np.nan)   
            else:
                antecedentes_padre.append(value.group(1)) 
        else:
            antecedentes_padre.append(value.group(1))
    else:
        antecedentes_padre.append(value.group(1))

In [74]:
df['Opinión plan'] = df['Opinión plan'].str.lower().str.strip()

In [75]:
igE = []
for sentence in df['Opinión plan'].values:
    if type(sentence) == str:
        value = re.search(r'\paciente con ap\b', sentence)
    else: 
        igE.append(np.nan)
        continue
    if value is None:  
        value = re.search(r'\se solicita ig e\b', sentence)
        if value is None:  
            value = re.search(r'\se solicit ig e\b', sentence)
            if value is None:  
                igE.append('Sin dato')
            else:
                igE.append('No')
        else:
            igE.append('No')
    else:
        igE.append('Si')

In [76]:
df1 = df['Motivo consulta'].astype(str) + ' ' + df['Enfermedad actual'].astype(str) + ' ' + df['Revisión sistemas'].astype(str) + ' ' + df['Antecedentes'].astype(str) + ' ' + df['Examen físico'].astype(str) + ' ' + df['Opinión plan'].astype(str)

df1 = df1.tolist()

In [77]:
result = np.array([df['Id Paciente'].values,df['Fecha_inicio_consulta'], df['Fecha nacimiento'], generic_1('peso', df['Examen físico']), generic('talla', df['Examen físico']), gestacional, tipo_parto, clas_eg, peso_nacer, fumador, antecedentes_madre, antecedentes_padre, igE, df1])
result = np.stack(result).T
df_salida = pd.DataFrame(result, columns=['id', 'fecha_consulta', 'fecha_nacimiento', 'peso', 'talla', 'gestacional', 'tipo_parto', 'clas_eg', 'peso_nacer', 'fumador', 'antecedentes_madre', 'antecedentes_padre', 'igE', 'nlp'])

In [79]:
df_salida.to_excel('Excel/F508_F_Procesed_.xlsx', index=False)

# PDF

In [22]:
# importing required modules
import PyPDF2

# creating a pdf file object
pdfFileObj = open('dictionary-2011_english_pdf-version.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# printing number of pages in pdf file
print(pdfReader.numPages)

pages = []

for i in range(8,9):
    # creating a page object
    pageObj = pdfReader.getPage(i)

    # extracting text from page
    pages.append(pageObj.extractText())

# closing the pdf file object
pdfFileObj.close()


Xref table not zero-indexed. ID numbers for objects will be corrected.


61


In [23]:
extracted = []
for i in pages:
    extracted.append(i.split('\n'))
extracted = [item for sublist in extracted for item in sublist]

In [129]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Error loading spanish: Package 'spanish' not found in
[nltk_data]     index


In [171]:
from nltk.corpus import stopwords
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [159]:
f = open("dictionary-2011_english_pdf-version.txt", "r")
lines = f.readlines()
f.close()

In [23]:
extract = []
for i in lines:
    extract.append(re.split(r'\s{2,}', i.replace('\n\n', '\n').replace('\n', ' ')))

In [161]:
extract = [item for sublist in extract for item in sublist]
extracted = []
for i in extract:
    extracted.append(i.split('-'))
extracted = [item for sublist in extracted for item in sublist]
extracted_1 = []
for i in extracted:
    extracted_1.append(i.split(','))
extracted_1 = [str.strip(item) for sublist in extracted_1 for item in sublist]
extracted_2 = []
for i in extracted_1:
    extracted_2.append(i.split('/'))
extracted_2 = [str.strip(item) for sublist in extracted_2 for item in sublist]
extracted_3 = []
for i in extracted_2:
    extracted_3.append(i.split(' '))
extracted_3 = [str.strip(item) for sublist in extracted_3 for item in sublist]


In [162]:
extracted_4 = []
for i in extracted_3:
    extracted_4.append(i.split(' '))
extracted_4 = [str.strip(item) for sublist in extracted_4 for item in sublist]

In [163]:
extracted_4 = np.array(extracted_4)
extracted_4 = extracted_4[np.where(extracted_4 != '')]
extracted_4 = np.unique(extracted_4)
extracted_4[np.where(extracted_4 == 'eritema')]

array(['eritema'], dtype='<U20')

In [164]:
extracted_4[np.where(extracted_4 == 'urticaria')]

array(['urticaria'], dtype='<U20')

In [165]:
final_dict = []
for w in extracted_4:
    if not w.lower() in words:
        final_dict.append(w)

In [186]:
stopwords = nltk.corpus.stopwords.words("spanish")
def stop(x):
    if x not in stopwords:
        if len(x) > 2:
            if x != '':
                return x 
    else:
        np.nan

In [187]:
diccionario = pd.DataFrame(extracted_4, columns=['term'])
diccionario['term'] = diccionario['term'].apply(lambda x: stop(x))
diccionario.to_excel('diccionario.xlsx')

In [33]:
f = open("English-Spanish-Dictionary-of-Health-Related-Terms.txt", "r", encoding="ansi")
lines = f.readlines()
f.close()

In [34]:
lines

['abortion-therapeutic abruptio placentae abscess\n',
 'absent periods absorption\n',
 'abstinence\n',
 'abuse\n',
 'accelerated reaction\n',
 'acetaminophen ache\n',
 'Achilles tendon acidosis\n',
 'acne\n',
 'action plan\n',
 'active/inactive germ acute\n',
 "Adam's apple\n",
 'addict\n',
 'addiction addictive adenoiditis adhesion adhesive tape adipose tissue\n',
 '\n',
 'adrenal gland adrenaline advanced infection affected\n',
 'afflicted\n',
 'afraid\n',
 'African American\n',
 'after pains\n',
 'T~RMINOS  EN ORDEN ALFABETICO\n',
 '\n',
 'las 24 horas  del dia, los 7 dias de la semana\n',
 'vision perfecta\n',
 '\n',
 '\n',
 'abdomen, est~mago, barriga, panza anormal, irregular\n',
 'excremento de apariencia anormal:\n',
 '-  negro\n',
 '-  con sangre\n',
 '-  de color arcilloso\n',
 '- grasiento\n',
 '-  duro\n',
 '-  suelto, aguado\n',
 '-  con moco\n',
 'aborto espontaneo/ natural/ no deseado, p~rdida, perder el beb~, mal parto, brote, cama chica, contrariedad, mala cama, descar

# Sintomas

In [52]:
diccionario = pd.read_csv('diccionario.csv')

In [53]:
re.findall(r"\b(?=("+'|'.join(diccionario.term.values[np.where(diccionario.term.values!='nan')])+r'\b'"))", df['Motivo consulta'].values[4])

['product',
 'pediatra',
 'adversa',
 'urticaria',
 'eritema',
 'facial',
 'sibilante']

In [112]:
sintoma_persona = []
for i in df['Motivo consulta'].values:
    sintoma_persona.append(re.findall(r"\b(?=("+'|'.join(diccionario.term.values[np.where(diccionario.term.values!='nan')])+r'\b'"))", i))

In [113]:
sintomas = np.zeros((len(sintoma_persona),len(max(sintoma_persona, key=len))), dtype='<U20')
sintomas[:] = ""

In [114]:
for i in range(len(sintoma_persona)):
    if len(sintoma_persona[i]) < 24:
        for j in range(24-len(sintoma_persona[i])):
            sintoma_persona[i].append("")

In [74]:
result = np.array([df['Id Paciente'].values, generic_1('peso', df['Examen físico']), generic('talla', df['Examen físico']), gestacional, tipo_parto, clas_eg, peso_nacer, fumador, antecedentes_madre, antecedentes_padre, igE])
result = np.stack(result).T
result

array([['6538955', '8310', '71', ...,
        'aergia polen s de alergia oral (manzana- apio-zanahoria-nueces)',
        'rinitis alergica: polen -perro-gatos-acaros. banano- piña.',
        'Sin dato'],
       ['71743330', '82.5', '178', ..., 'asma', 'no atopia', 'Sin dato'],
       ['88282652', '35.7', '143', ..., 'no atopia', 'no atopia',
        'Sin dato'],
       ...,
       ['1239490297', '11.360', '81', ..., 'nan', 'no atopia.',
        'Sin dato'],
       ['6179278418', '8.7', '71', ..., 'nan', 'nan', 'Sin dato'],
       ['6179278418', '5870', '57', ..., 'nan', 'nan', 'Sin dato']],
      dtype='<U198')

In [75]:
df_peso_talla = pd.DataFrame( result, columns=['id', 'peso', 'talla', 'gestacional', 'tipo_parto', 'clas_eg', 'peso_nacer', 'fumador', 'antecedentes_madre', 'antecedentes_padre', 'igE'])
df_peso_talla

,id,peso,talla,gestacional,tipo_parto,clas_eg,peso_nacer,fumador,antecedentes_madre,antecedentes_padre,igE
0,6538955,8310,71,nan,Vaginal,Termino,nan,nan,aergia polen s de alergia oral (manzana- apio-...,rinitis alergica: polen -perro-gatos-acaros. b...,Sin dato
1,71743330,82.5,178,nan,Cesarea,Pretermino,nan,No es fumador pasivo,asma,no atopia,Sin dato
2,88282652,35.7,143,nan,Cesarea,Termino,nan,No es fumador pasivo,no atopia,no atopia,Sin dato
3,540705758,18.4,114,nan,Vaginal,Pretermino,nan,No es fumador pasivo,no atopia,no atopia hermana: no atopia,Sin dato
4,560338861,9.8,82,32,Cesarea,Pretermino,nan,No es fumador pasivo,nan,rinitis madre. rinitis.,Si
...,...,...,...,...,...,...,...,...,...,...,...
613,1239088393,8360,70,nan,Vaginal,Pretermino,nan,nan,asma en la infancia. rinitis,rinitis,Sin dato
614,1239489705,8.650,76,nan,Cesarea,Termino,nan,No es fumador pasivo,no atopia,asma bornquial,Sin dato
615,1239490297,11.360,81,nan,Cesarea,Pretermino,nan,No es fumador pasivo,nan,no atopia.,Sin dato
616,6179278418,8.7,71,nan,Cesarea,Pretermino,nan,No es fumador pasivo,nan,nan,Sin dato


In [48]:
df_peso_talla.to_excel('Base inicial.xlsx')

In [117]:
length = max(map(len, sintoma_persona))
y=np.array([xi+[None]*(length-len(xi)) for xi in sintoma_persona])
df_sintomas = pd.DataFrame(y)
df_sintomas.to_excel('sintomas.xlsx')

In [205]:
from collections import Counter

a = dict(Counter(df['Identificacion']))

In [206]:
from collections import Counter

words = ['a', 'b', 'c', 'a']

Counter(df['Identificacion']).keys() # equals to list(set(words))
Counter(Counter(df['Identificacion']).values()) # counts the elements' frequency

Counter({1: 259, 2: 49, 3: 22, 4: 6, 5: 4})

In [208]:
49+(22*2)+(18)+16

127

# Frecuencias

In [14]:
df = pd.read_csv('BASE DE DATOS INGIENERO.csv')
print(df.shape)
df.drop_duplicates(subset=['ID'], inplace=True)
print(df.shape)

(488, 17)
(488, 17)


In [15]:
df.columns

Index(['ID', 'Fecha de nacimiento', 'Fecha consulta', 'Sexo', 'Peso (kg)',
       'Talla (cm)', 'WAZ', 'HAZ', 'BAZ', 'BMI', 'WHZ', 'Edad', 'Edad meses',
       'haz_clas', 'baz_clas', 'waz_clas', 'whz_clas'],
      dtype='object')

In [16]:
df.dropna(subset=['Peso (kg)', 'Talla (cm)'], inplace=True)
print(df.shape)
df = df[df.Edad < 19]
print(df.shape)

(488, 17)
(485, 17)


In [17]:
def clas_edad(y, m):
    if y == 0:
        if m < 6:
            return "Menor a seis meses"
        if m >= 6:
            return "Mayor a seis meses"
    elif y <= 2:
        return "De uno a dos años" 
    elif y <= 4:
        return "De tres a cuatro años" 
    elif y <= 10:
        return "De cinco a diez años" 
    elif y <= 15:
        return "De diez a 15 años" 
    elif y <= 18:
        return "De 15 a 18 años" 
    else:
        return 'No data'

In [18]:
df['edad_clas'] = df.apply(lambda x: clas_edad(x.Edad, x['Edad meses']), axis = 1)

In [19]:
df['edad_clas'].value_counts()

De cinco a diez años     194
De tres a cuatro años    145
De uno a dos años         62
De diez a 15 años         60
De 15 a 18 años           20
Mayor a seis meses         4
Name: edad_clas, dtype: int64

In [20]:
def haz_clas(x):
    print(x)
    if x >= -1:
        return "Talla Adecuada para la Edad"
    elif x >= -2:
        return "Riesgo de Talla Baja"
    elif x < -2:
        return "Talla baja para la edad"
    else:
        return "No data"
def baz_clas(x, edad):
    if edad < 5:
        if x > 3:
            return "Obesidad"
        elif x > 2:
            return "Sobrepeso"
        elif x > 1:
            return "Riesgo de sobrepeso"
        elif x <= 1:
            return "No aplica (Verificar con P/T)"
        else:
            return "No data"
    else:
        if x > 2:
            return "Obesidad"
        elif x > 1:
            return "Sobrepeso"
        elif x >= -1:
            return "Adecuado"
        elif x >= -2:
            return "Riesgo de delgadez"
        elif x < -2:
            return "Delgadez"
        else:
            return "No data"
def whz_clas(x, edad):
    if edad < 5:
        if x > 3:
            return "Obesidad"
        elif x > 2:
            return "Sobrepeso"
        elif x > 1:
            return "Riesgo de Sobrepeso"
        elif x >= -1:
            return "Peso Adecuado para la Talla"
        elif x >= -2:
            return "Riesgo de Desnutricion Aguda"
        elif x >= -3:
            return "Desnutricion Aguda Moderada"
        elif x < -3:
            return "Desnutricion Aguda Severa"
        else:
            return "No data"
    else:
        return "Mayor a 5"
def waz_clas(x, edad):
    if edad < 5:
        if x > 1:
            return "No aplica (Verificar con IMC/E)"
        elif x >= -1:
            return "Peso adecuado para la edad"
        elif x >= -2:
            return "Riesgo de desnutricion"
        elif x < -2:
            return "Desnutricion"
        else:
            return "No data"
    else:
        return "Mayor a 5"

In [21]:
df['haz_clas'] = df.apply(lambda x: haz_clas(x.HAZ), axis = 1)
df['baz_clas'] = df.apply(lambda x: baz_clas(x.BAZ, x.Edad), axis = 1)
df['waz_clas'] = df.apply(lambda x: waz_clas(x.WAZ, x.Edad), axis = 1)
df['whz_clas'] = df.apply(lambda x: whz_clas(x.WHZ, x.Edad), axis = 1)

0.27
-0.64
-0.39
-0.88
-0.04
-1.05
0.06
-1.03
-0.41
-0.75
-0.66
0.05
-0.02
-0.19
0.85
-0.49
1.68
-0.59
-2.34
0.81
1.54
1.13
-0.39
-0.42
-0.04
1.78
1.11
-1.08
0.44
-0.31
-1.06
-0.59
2.49
-1.74
0.11
-0.68
1.38
-0.98
-0.05
1.62
-1.07
1.08
0.94
1.23
0.11
-0.66
0.23
0.16
-1.14
-0.18
0.41
0.6
-1.85
0.21
0.0
0.9
0.41
0.24
-1.55
1.39
0.08
-0.08
0.0
1.44
-0.7
-0.44
1.48
0.02
0.76
-1.36
-0.88
0.32
-0.74
2.93
1.58
-1.59
-1.3
0.33
-0.29
2.85
0.51
-0.94
1.9
-0.34
0.28
-0.93
-1.69
0.36
1.28
0.41
1.2
0.42
-0.12
0.24
0.07
0.83
2.1
0.85
-0.47
-0.19
1.82
0.06
0.68
-0.33
-0.96
1.33
1.33
0.69
-0.36
0.54
-1.06
-1.1
0.09
0.8
-0.14
-0.16
0.58
-1.68
1.06
0.64
1.05
-0.09
0.12
0.49
-1.74
0.72
-0.36
2.31
-0.75
-0.18
-0.22
-0.7
0.86
0.5
0.83
-0.72
-1.47
-0.98
-0.86
-1.37
-0.81
-0.15
-0.29
1.59
-1.18
1.56
-0.31
1.55
1.68
1.97
1.22
-0.63
1.35
0.9
-0.98
-0.36
-1.98
1.47
-1.36
-0.01
0.19
-0.55
-1.21
-1.33
0.68
1.31
0.06
0.13
1.11
1.28
1.2
0.64
-1.52
1.84
-0.36
-0.38
1.75
-0.61
-1.59
-0.44
-1.07
2.16
-0.43
0.65
-0.3
-

In [22]:
(df['haz_clas'].value_counts() / df.shape[0]) * 100

Talla Adecuada para la Edad    88.659794
Riesgo de Talla Baja           10.309278
Talla baja para la edad         1.030928
Name: haz_clas, dtype: float64

In [23]:
(df['baz_clas'].value_counts() / df.shape[0]) * 100

No aplica (Verificar con P/T)    35.257732
Adecuado                         32.989691
Sobrepeso                        12.989691
Riesgo de sobrepeso               5.773196
Obesidad                          4.948454
Riesgo de delgadez                4.948454
Delgadez                          3.092784
Name: baz_clas, dtype: float64

In [24]:
(df['waz_clas'].value_counts() / df.shape[0]) * 100

Mayor a 5                          56.494845
Peso adecuado para la edad         35.257732
No aplica (Verificar con IMC/E)     5.360825
Riesgo de desnutricion              2.474227
Desnutricion                        0.412371
Name: waz_clas, dtype: float64

In [25]:
(df['whz_clas'].value_counts() / df.shape[0]) * 100

Mayor a 5                       56.494845
Peso Adecuado para la Talla     30.309278
Riesgo de Sobrepeso              5.360825
Riesgo de Desnutricion Aguda     4.123711
Sobrepeso                        1.237113
Obesidad                         1.030928
Desnutricion Aguda Moderada      0.824742
No data                          0.412371
Desnutricion Aguda Severa        0.206186
Name: whz_clas, dtype: float64

In [38]:
state_pcts = df.groupby(['edad_clas', 'waz_clas',]).agg({'ID': 'count'})
state_pcts = state_pcts.rename(columns={'ID':'Cantidad'})
state_pcts['Porcentaje'] = state_pcts.groupby(level=0).apply(lambda x:
                                                 round(100 * x / float(x.sum()), 2))
state_pcts

Cantidad  Porcentaje
edad_clas             waz_clas                                             
De 15 a 18 años       Mayor a 5                              20      100.00
De cinco a diez años  Mayor a 5                             194      100.00
De diez a 15 años     Mayor a 5                              60      100.00
De tres a cuatro años Desnutricion                            1        0.69
                      No aplica (Verificar con IMC/E)        15       10.34
                      Peso adecuado para la edad            119       82.07
                      Riesgo de desnutricion                 10        6.90
De uno a dos años     Desnutricion                            1        1.61
                      No aplica (Verificar con IMC/E)        11       17.74
                      Peso adecuado para la edad             48       77.42
                      Riesgo de desnutricion                  2        3.23
Mayor a seis meses    Peso adecuado para la edad              4      100.00

In [39]:
state_pcts = df.groupby(['edad_clas', 'baz_clas',]).agg({'ID': 'count'})
state_pcts = state_pcts.rename(columns={'ID':'Cantidad'})
state_pcts['Porcentaje'] = state_pcts.groupby(level=0).apply(lambda x:
                                                 round(100 * x / float(x.sum()), 2))
state_pcts

Cantidad  Porcentaje
edad_clas             baz_clas                                           
De 15 a 18 años       Adecuado                             12       60.00
                      Obesidad                              3       15.00
                      Riesgo de delgadez                    2       10.00
                      Sobrepeso                             3       15.00
De cinco a diez años  Adecuado                            113       58.25
                      Delgadez                              8        4.12
                      Obesidad                             15        7.73
                      Riesgo de delgadez                   19        9.79
                      Sobrepeso                            39       20.10
De diez a 15 años     Adecuado                             35       58.33
                      Delgadez                              7       11.67
                      Obesidad                              2        3.33
                      Riesgo de delgadez                    3        5.00
                      Sobrepeso                            13       21.67
De tres a cuatro años No aplica (Verificar con P/T)       119       82.07
                      Obesidad                              2        1.38
                      Riesgo de sobrepeso                  20       13.79
                      Sobrepeso                             4        2.76
De uno a dos años     No aplica (Verificar con P/T)        48       77.42
                      Obesidad                              2        3.23
                      Riesgo de sobrepeso                   8       12.90
                      Sobrepeso                             4        6.45
Mayor a seis meses    No aplica (Verificar con P/T)         4      100.00

In [40]:
state_pcts = df.groupby(['edad_clas', 'haz_clas',]).agg({'ID': 'count'})
state_pcts = state_pcts.rename(columns={'ID':'Cantidad'})
state_pcts['Porcentaje'] = state_pcts.groupby(level=0).apply(lambda x:
                                                 round(100 * x / float(x.sum()), 2))
state_pcts

Cantidad  Porcentaje
edad_clas             haz_clas                                         
De 15 a 18 años       Riesgo de Talla Baja                2       10.00
                      Talla Adecuada para la Edad        17       85.00
                      Talla baja para la edad             1        5.00
De cinco a diez años  Riesgo de Talla Baja               18        9.28
                      Talla Adecuada para la Edad       175       90.21
                      Talla baja para la edad             1        0.52
De diez a 15 años     Riesgo de Talla Baja                6       10.00
                      Talla Adecuada para la Edad        54       90.00
De tres a cuatro años Riesgo de Talla Baja               13        8.97
                      Talla Adecuada para la Edad       131       90.34
                      Talla baja para la edad             1        0.69
De uno a dos años     Riesgo de Talla Baja               11       17.74
                      Talla Adecuada para la Edad        49       79.03
                      Talla baja para la edad             2        3.23
Mayor a seis meses    Talla Adecuada para la Edad         4      100.00

In [41]:
state_pcts = df.groupby(['edad_clas', 'whz_clas',]).agg({'ID': 'count'})
state_pcts = state_pcts.rename(columns={'ID':'Cantidad'})
state_pcts['Porcentaje'] = state_pcts.groupby(level=0).apply(lambda x:
                                                 round(100 * x / float(x.sum()), 2))
state_pcts

Cantidad  Porcentaje
edad_clas             whz_clas                                          
De 15 a 18 años       Mayor a 5                           20      100.00
De cinco a diez años  Mayor a 5                          194      100.00
De diez a 15 años     Mayor a 5                           60      100.00
De tres a cuatro años Desnutricion Aguda Moderada          4        2.76
                      Desnutricion Aguda Severa            1        0.69
                      No data                              2        1.38
                      Obesidad                             3        2.07
                      Peso Adecuado para la Talla         98       67.59
                      Riesgo de Desnutricion Aguda        16       11.03
                      Riesgo de Sobrepeso                 18       12.41
                      Sobrepeso                            3        2.07
De uno a dos años     Obesidad                             2        3.23
                      Peso Adecuado para la Talla         46       74.19
                      Riesgo de Desnutricion Aguda         3        4.84
                      Riesgo de Sobrepeso                  8       12.90
                      Sobrepeso                            3        4.84
Mayor a seis meses    Peso Adecuado para la Talla          3       75.00
                      Riesgo de Desnutricion Aguda         1       25.00

In [32]:
df.shape

(485, 18)

In [33]:
df.shape

(485, 18)

In [34]:
df[df.BAZ < -2][df.Edad >= 5]

C:\Users\santy\AppData\Local\Temp\ipykernel_13796\2256080696.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.BAZ < -2][df.Edad >= 5]


,ID,Fecha de nacimiento,Fecha consulta,Sexo,Peso (kg),Talla (cm),WAZ,HAZ,BAZ,BMI,WHZ,Edad,Edad meses,haz_clas,baz_clas,waz_clas,whz_clas,edad_clas
24,1013461706,5/23/2008,2/25/2021,M,32.4,154.0,NaN,-0.04,-3.03,13.7,NaN,12.0,9,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De diez a 15 años
158,1023594701,4/28/2013,5/9/2022,M,20.3,124.5,-2.39,-1.36,-2.41,13.1,NaN,9.0,1,Riesgo de Talla Baja,Delgadez,Mayor a 5,Mayor a 5,De cinco a diez años
163,1025653662,9/19/2008,4/28/2021,M,29.3,143.5,NaN,-1.33,-2.43,14.2,NaN,11.0,7,Riesgo de Talla Baja,Delgadez,Mayor a 5,Mayor a 5,De diez a 15 años
183,1027741481,2/21/2009,3/9/2021,M,33.4,154.0,NaN,0.65,-2.34,14.1,NaN,12.0,1,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De diez a 15 años
213,1033183564,12/20/2007,7/19/2021,F,33.9,150.5,NaN,-1.16,-2.18,15.0,NaN,13.0,6,Riesgo de Talla Baja,Delgadez,Mayor a 5,Mayor a 5,De diez a 15 años
216,1033200095,4/17/2016,6/30/2022,M,16.8,114.5,-1.72,-0.53,-2.26,12.8,NaN,6.0,2,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De cinco a diez años
222,1033257612,4/23/2005,6/18/2020,M,43.7,167.0,NaN,-0.34,-2.30,15.7,NaN,15.0,1,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De diez a 15 años
224,1033263472,5/8/2013,1/18/2019,F,12.0,115.5,-3.64,0.43,-5.80,9.0,NaN,5.0,7,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De cinco a diez años
288,1035004630,9/25/2014,3/9/2022,F,18.8,122.0,-1.50,-0.25,-2.15,12.6,NaN,7.0,11,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De cinco a diez años
291,1035004821,10/28/2014,12/17/2020,M,16.5,114.0,-1.81,-0.56,-2.38,12.7,NaN,6.0,2,Talla Adecuada para la Edad,Delgadez,Mayor a 5,Mayor a 5,De cinco a diez años


In [35]:
df.to_excel('BASE INGENIERO ENVIO.xlsx', index=False)

In [112]:
df.Edad[df.Edad<5]

0      0
3      1
4      0
5      2
6      0
      ..
461    3
462    0
463    1
464    1
465    0
Name: Edad, Length: 276, dtype: int64